In [1]:
%load_ext autoreload
%autoreload 2
import torch
import pickle
from cot_probing.typing import *
from cot_probing.activations import (
    Activations,
    QuestionActivations,
    get_mean_activations_by_common_token,
)
from cot_probing.eval import EvalQuestion, get_common_tokens, get_correct_incorrect_idxs
from cot_probing.utils import get_train_test_split

In [2]:
with open("../hf_results/activations_google--gemma-2-9b_snarks_S0_N151.pkl", "rb") as f:
    activations = pickle.load(f)
eval_results = activations.eval_results
eval_questions = eval_results.questions

In [9]:
FREQ_THRESHOLD = 0.2
TRAIN_FRAC = 0.8
SEED = 0
correct_q_idxs, incorrect_q_idxs = get_correct_incorrect_idxs(eval_questions)
train_correct_idxs, test_correct_idxs = get_train_test_split(
    correct_q_idxs, TRAIN_FRAC, SEED
)
train_incorrect_idxs, test_incorrect_idxs = get_train_test_split(
    incorrect_q_idxs, TRAIN_FRAC, SEED
)
common_tokens = get_common_tokens(eval_questions, FREQ_THRESHOLD)
mean_activations_correct = get_mean_activations_by_common_token(
    common_tokens, activations, train_correct_idxs
)
mean_activations_incorrect = get_mean_activations_by_common_token(
    common_tokens, activations, train_incorrect_idxs
)

In [10]:
probes_by_token = {
    t: mean_activations_correct[t] - mean_activations_incorrect[t]
    for t in common_tokens
}